### 생성 데이터 컬럼
> 위치 및 충전기 정보
- 충전소위치  충전기 <br/>	
> 충전기 정보 상세
- 충전기타입  충전기상태  충전기용량  방전지원여부  충전량  요청충전량  방전량  요청방전량<br>  
> 시간 정보
- 연결시작시간  연결종료시간  충전시작시간  충전종료시간  출방예상시간<br/>
> 차량 Soc 정보
- 베터리용량  시작값  종료값  연비

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [100]:
#재생성 유지
random.seed(42)
np.random.seed(42)
#충전소위치
st_number = 50  # 충전소 개수
location = [f'st-{str(n).zfill(2)}' for n in range(st_number)]
#충전기 Electric Vehicle Supply Equipment
evse = []
for s in location:
    evse_n = random.randint(2,6)
    for i in range(evse_n):
        evse.append(f'{s}_evse-{str(i).zfill(2)}')

In [ ]:
#  충전기타입  충전기상태  충전기용량  방전지원여부  충전량  요청충전량  방전량  요청방전량